In [2]:
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_target = df['SUBCLASS']

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns=['ID'], inplace=True)
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [21]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('SUBCLASS').reset_index(name='tcounts')
diseaseList

,SUBCLASS,tcounts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [6]:
#td = pd.concat([data, df_test])
td = df_test
num_list = []
selected_genes2 = []
i = 0
th = len(td)*0.1
print(th)
for g in td.columns:
    c  = td[[g]].apply(lambda x: x=='WT').agg(sum).values[0]
    if c > th:
        num_list.append(c)
len(num_list)    
    


254.60000000000002


4384

In [2]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [3]:
#preprocessing for total
#for model csv
total = pd.concat([data,df_test])
f = lambda x : '' if x == 'WT' else x
total = total.map(f)



In [4]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,,,L305L,,N252I N251I,G679V,,
2544,,,,,,,,,R1517H,,...,,,,,,,,,,


In [5]:
disease_list = df.SUBCLASS.unique()
disease_list

array(['KIPAN', 'SARC', 'SKCM', 'KIRC', 'GBMLGG', 'STES', 'BRCA', 'THCA',
       'LIHC', 'HNSC', 'PAAD', 'OV', 'PRAD', 'UCEC', 'LAML', 'COAD',
       'ACC', 'LGG', 'LUSC', 'LUAD', 'CESC', 'PCPG', 'THYM', 'BLCA',
       'TGCT', 'DLBC'], dtype=object)

In [7]:
### ###
nCols = len(data.columns)
from collections import defaultdict
cDic = defaultdict()
for i, c in enumerate(data.columns):
    cDic[i] = c

In [8]:
def make_ndf (data, df_test):
    #data.drop('SUBCLASS', axis=1,inplace=True)
    #target = pd.concat([data,df_test])
    
    #2칸=>1칸
    ndf = pd.DataFrame()
    
    for i in range(nCols//2):
        ndf[cDic[i*2]] = total[total.columns[i*2]] +' '+ total[total.columns[(i*2+1)]]
    return ndf

In [ ]:
#preprocessing
#for writing csv
def preprocessing_for_writing():
    f = lambda x : '' if x == 'WT' else x
    tdf = data.map(f)
    tdf['SUBCLASS'] = df.loc[:, df.columns[1:2]]
    tdf


In [456]:
df.loc[:, df.columns[:2]]

,ID,SUBCLASS
0,TRAIN_0000,KIPAN
1,TRAIN_0001,SARC
2,TRAIN_0002,SKCM
3,TRAIN_0003,KIRC
4,TRAIN_0004,GBMLGG
...,...,...
6196,TRAIN_6196,LUAD
6197,TRAIN_6197,LGG
6198,TRAIN_6198,COAD
6199,TRAIN_6199,TGCT


In [459]:
def getnDF(d):
    #2칸=>1칸
    ndf = pd.DataFrame()
    target = tdf[tdf['SUBCLASS']==d]
    target.drop('SUBCLASS', axis=1,inplace=True)
    for i in range(nCols//2):
        ndf[cDic[i*2]] = target[target.columns[i*2]] +' '+ target[target.columns[(i*2+1)]]
    return ndf

In [467]:
#for writing
for d in tdf['SUBCLASS'].unique():
    ndf = getnDF(d)
    for c in ndf.columns:
        ndf[c] = ndf[c].apply(lambda x : '-' if (x =='' or x==' ') else 'X')
    ndf.to_csv('./data1/bstr-'+d+'-1.csv')


In [308]:
a = [1,2,3,4,5,6,7,8]
for i in range (0,len(a)//2):
    print(i*2, (i+1)*2)

0 2
2 4
4 6
6 8


In [398]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in range(10): #range(nCols//2):
    print(i, i+1) 
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  


0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10


In [403]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in range(10): #range(nCols//2):
    print(i*2, i*2 +1) 
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  


0 1
2 3
4 5
6 7
8 9
10 11
12 13
14 15
16 17
18 19


In [404]:
#2칸=>1칸
ndf = pd.DataFrame()
for i in  range(nCols//2):
    ndf[cDic[i*2]] = data[data.columns[i*2]] +' '+ data[data.columns[(i*2+1)]]
    #print(i*2, (i+1) * 2) 
    #strr = data[data.columns[i*2]] data[data.columns[ (i+1) * 2]]
    #ndf[cDic[i*2]] =  data[data.columns[ i*2] or data.columns[  (i+1) * 2]]  
ndf


,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [405]:
ndf

,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [426]:
ndf

,A2M,AADAT,ABAT,ABCA2,ABCA4,ABCA6,ABCA9,ABCB11,ABCB6,ABCB8,...,ZC3H14,ZFP36,ZFPM2,ZMIZ1,ZNF185,ZNF292,ZNF639,ZNFX1,ZPBP,ZWINT
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,T426T,,,...,,,,,,,,,,
3,,,,,,,,,,,...,Y904H,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,L101M,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,,T181S
6199,,,,,,,,,,,...,,,,,,,,,,


In [438]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX,SUBCLASS
0,,,,,,,,,,,...,,,,,,,,,,KIPAN
1,,,,,,,,,,,...,,,,,,,,,,SARC
2,R895R,,,,,,,,,,...,,,,,,,,,,SKCM
3,,,,,,,,,,,...,,,,,,,,,,KIRC
4,,,,,,,,,,,...,,,,,,,,,,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,,...,,,,,,,,,,LUAD
6197,,,,,,,,,,,...,,,,,,,,,,LGG
6198,,,,,,,,,,,...,,,,,,,,T181S,,COAD
6199,,,,,,,,,,,...,,,,,,,,,,TGCT


In [439]:
data[data['SUBCLASS']=='BRCA'].drop('SUBCLASS', axis=1)

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
6,,,,,,,,,,,...,,,,,,,,,,
18,,,,,,,,,,,...,,,,,,,,,,
26,,,,,,,,,,,...,,,,,,,,,,
28,,,,,,,,,,,...,,,,,,,,,,
39,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,,,,,,,,,,,...,,,,,,,,,,
6162,,,,,,,,,,,...,,,,,,,,,,
6165,,,,,,,,,,,...,,,,,,,,,,
6179,,,,,,,,,,,...,,,,,,,,,,


In [411]:
#df[df.columns[1::2]]
#df[df.columns[0::2]]

In [3]:
df[df.ABCB8 !='WT']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
14,TRAIN_0014,SKCM,WT,WT,WT,WT,WT,F157L S1042F,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
354,TRAIN_0354,DLBC,A615A,WT,WT,WT,WT,R1228R P2150P,WT,V150V,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
468,TRAIN_0468,OV,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
489,TRAIN_0489,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
532,TRAIN_0532,LUSC,WT,WT,WT,WT,WT,WT,R2400W A903S,WT,...,E2679*,WT,WT,WT,WT,WT,WT,WT,WT,WT
562,TRAIN_0562,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
717,TRAIN_0717,STES,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,Y46H,WT,WT
843,TRAIN_0843,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,S1244N,WT,WT,WT,WT,WT
1138,TRAIN_1138,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1369,TRAIN_1369,SKCM,E1311K E342K R147C E1311K E342K R147C,P307S T163T P307S T163T,P244L P244L,WT,P239S P239S,P91S P91S,S1492F A1262A F1948F S1237F L1683L S1492F A126...,G495G P72L L310L D1089D G495G P72L L310L D1089D,...,P873S L2423V R2173* S1473F P873S L2423V R2173*...,E54K E54K,WT,WT,F380F S197F F380F S197F,G378S L264L P349P G121I I207I G378S L264L P349...,E143K E143K,L489L L489L,WT,WT


In [14]:
from sklearn.cluster import DBSCAN
import numpy as np

# 주어진 배열을 numpy 배열로 변환
data = np.array([189, 180, 18, 16, 5, 5, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1]).reshape(-1, 1)

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=3, min_samples=2)
labels = dbscan.fit_predict(data)

# 각 데이터가 속한 클러스터 출력
print(labels)


[-1 -1  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1]


In [15]:
from sklearn.cluster import KMeans
import numpy as np

# 주어진 배열을 numpy 배열로 변환
data = np.array([189, 180, 18, 16, 5, 5, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1]).reshape(-1, 1)

# K-Means 클러스터링 (k=2)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(data)

# 각 데이터가 속한 클러스터 출력
print(kmeans.labels_)


[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('SUBCLASS').reset_index(name='tcounts')
diseaseList

,SUBCLASS,tcounts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [268]:
consist = df[df.RAPGEF6 !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
merged.dropna(inplace=True)
merged['occupied_rate'] = merged['gcount']/merged['tcounts']
merged=merged.sort_values(by='occupied_rate',ascending=False)
merged

,SUBCLASS,tcounts,gcount,occupied_rate
25,UCEC,198,13.0,0.065657
20,SKCM,276,10.0,0.036232
4,COAD,223,8.0,0.035874
21,STES,379,12.0,0.031662
1,BLCA,104,3.0,0.028846
13,LUAD,184,5.0,0.027174
7,HNSC,223,6.0,0.026906
3,CESC,155,3.0,0.019355
12,LIHC,158,3.0,0.018987
9,KIRC,334,6.0,0.017964


In [186]:

consist = df[df.SYNE1 !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
merged.dropna(inplace=True)
merged['occupied_rate'] = merged['gcount']/merged['tcounts']
merged=merged.sort_values(by='occupied_rate',ascending=False)
merged

,SUBCLASS,tcounts,gcount,occupied_rate
14,LUSC,178,61,0.342697
21,STES,379,116,0.306069
20,SKCM,276,84,0.304348
4,COAD,223,62,0.278027
5,DLBC,38,10,0.263158
1,BLCA,104,22,0.211538
7,HNSC,223,43,0.192825
3,CESC,155,24,0.154839
13,LUAD,184,28,0.152174
25,UCEC,198,27,0.136364


In [1]:
consist = df[df.PDK4 !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
merged.dropna(inplace=True)
merged['occupied_rate'] = merged['gcount']/merged['tcounts']
merged=merged.sort_values(by='occupied_rate',ascending=False)
merged

NameError: name 'df' is not defined

In [205]:
consist = df[df.PDK4 !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
merged.dropna(inplace=True)
merged['occupied_rate'] = merged['gcount']/merged['tcounts']
merged=merged.sort_values(by='occupied_rate',ascending=False)
merged

AttributeError: 'DataFrame' object has no attribute 'PGRMC1'

In [181]:
consist = df[df.BRCA1 !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
merged.dropna(inplace=True)
merged['occupied_rate'] = merged['gcount']/merged['tcounts']
merged=merged.sort_values(by='occupied_rate',ascending=False)
merged

,SUBCLASS,tcounts,gcount,occupied_rate
5,DLBC,38,3.0,0.078947
25,UCEC,198,13.0,0.065657
14,LUSC,178,10.0,0.056180
20,SKCM,276,15.0,0.054348
21,STES,379,20.0,0.052770
7,HNSC,223,9.0,0.040359
13,LUAD,184,7.0,0.038043
15,OV,253,9.0,0.035573
3,CESC,155,5.0,0.032258
12,LIHC,158,5.0,0.031646


In [9]:
import pandas as pd 
df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [17]:
diseaseList

,disease,counts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [14]:
consist = df[df['SYNE1'] !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')

In [16]:
consist

,SUBCLASS,gcount
0,STES,116
1,SKCM,84
2,COAD,62
3,LUSC,61
4,BRCA,46
5,HNSC,43
6,LUAD,28
7,UCEC,27
8,CESC,24
9,GBMLGG,23


In [19]:
def explore_genes_distribution(gene_name):
    consist = df[df[gene_name] !='WT']['SUBCLASS'].value_counts().reset_index(name='gcount')
    merged = pd.merge(diseaseList, consist, on='SUBCLASS', how='outer')
    merged.dropna(inplace=True)
    merged['occupied_rate'] = merged['gcount']/merged['tcounts']
    merged=merged.sort_values(by='occupied_rate',ascending=False)
    print(merged)
    print("at test, ", df_test[gene_name].apply(lambda x : x!='WT').agg(sum))

In [22]:
genen = 'SYNE1'
explore_genes_distribution (genen)

   SUBCLASS  tcounts  gcount  occupied_rate
14     LUSC      178      61       0.342697
21     STES      379     116       0.306069
20     SKCM      276      84       0.304348
4      COAD      223      62       0.278027
5      DLBC       38      10       0.263158
1      BLCA      104      22       0.211538
7      HNSC      223      43       0.192825
3      CESC      155      24       0.154839
13     LUAD      184      28       0.152174
25     UCEC      198      27       0.136364
12     LIHC      158      14       0.088608
0       ACC       72       5       0.069444
2      BRCA      786      46       0.058524
16     PAAD      120       7       0.058333
6    GBMLGG      461      23       0.049892
18     PRAD      266      10       0.037594
9      KIRC      334      12       0.035928
19     SARC      198       7       0.035354
8     KIPAN      515      18       0.034951
15       OV      253       8       0.031621
11      LGG      229       7       0.030568
22     TGCT      124       2    

In [45]:
genen = 'MAST4'
explore_genes_distribution (genen)

   SUBCLASS  tcounts  gcount  occupied_rate
5      DLBC       38     4.0       0.105263
25     UCEC      198    16.0       0.080808
20     SKCM      276    22.0       0.079710
1      BLCA      104     7.0       0.067308
3      CESC      155     9.0       0.058065
14     LUSC      178     9.0       0.050562
21     STES      379    18.0       0.047493
4      COAD      223    10.0       0.044843
7      HNSC      223     8.0       0.035874
13     LUAD      184     5.0       0.027174
8     KIPAN      515    11.0       0.021359
9      KIRC      334     7.0       0.020958
16     PAAD      120     2.0       0.016667
0       ACC       72     1.0       0.013889
6    GBMLGG      461     6.0       0.013015
19     SARC      198     2.0       0.010101
2      BRCA      786     7.0       0.008906
11      LGG      229     1.0       0.004367
15       OV      253     1.0       0.003953
at test,  187


In [46]:
qq = ['AHNAK', 'TCHH', 'APC', 'CACNA1A', 'CPEB2', 'MAGEE1', 'TP53', 'KDM6B', 'TUBGCP6', 'KMT2D', 'HLA-A', 'UBC']
#{'AHNAK', 'APC', 'CACNA1A', 'DOCK4', 'POLR2A', 'STAB1', 'TCHH', 'TP53'}
for q in qq:
    print(q)
    explore_genes_distribution(q)

AHNAK
   SUBCLASS  tcounts  gcount  occupied_rate
20     SKCM      276    51.0       0.184783
14     LUSC      178    30.0       0.168539
1      BLCA      104    16.0       0.153846
13     LUAD      184    28.0       0.152174
21     STES      379    45.0       0.118734
7      HNSC      223    24.0       0.107623
25     UCEC      198    21.0       0.106061
4      COAD      223    18.0       0.080717
3      CESC      155     9.0       0.058065
5      DLBC       38     2.0       0.052632
12     LIHC      158     8.0       0.050633
19     SARC      198     9.0       0.045455
16     PAAD      120     5.0       0.041667
15       OV      253    10.0       0.039526
2      BRCA      786    24.0       0.030534
8     KIPAN      515    15.0       0.029126
17     PCPG      147     4.0       0.027211
18     PRAD      266     7.0       0.026316
6    GBMLGG      461    11.0       0.023861
22     TGCT      124     2.0       0.016129
9      KIRC      334     5.0       0.014970
23     THCA      324     3

In [ ]:
df_test = df_test.loc[:, list(data.columns)]

In [ ]:
   SLIT2     11
306   ARID4B     11
14    ABCB11     11
12     ABCA9     11
4062     TNC     11

In [ ]:
# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test2)

# 모델 성능 평가
accuracy = accuracy_score(y_test2, y_pred)
conf_matrix = confusion_matrix(y_test2, y_pred)
class_report = classification_report(y_test2, y_pred)

print(f'Accuracy: {accuracy}')
#print('Confusion Matrix:')
#print(conf_matrix)
print('Classification Report:')
print(class_report)

Accuracy: 0.5
Classification Report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           5       1.00      0.50      0.67        38
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         0
          25       0.00      0.00      0.00         0

    accuracy                           0.50        38
   macro avg       0.09      0.05      0.06        38
weighted avg       1.00      0.50      0.67        38



In [ ]:
# 테스트 데이터에 대한 예측
y_pred = clf.predict(X_test2)

# 모델 성능 평가
accuracy = accuracy_score(y_test2, y_pred)
conf_matrix = confusion_matrix(y_test2, y_pred)
class_report = classification_report(y_test2, y_pred)

print(f'Accuracy: {accuracy}')
#print('Confusion Matrix:')
#print(conf_matrix)
print('Classification Report:')
print(class_report)

Accuracy: 0.5
Classification Report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           5       1.00      0.50      0.67        38
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         0
          25       0.00      0.00      0.00         0

    accuracy                           0.50        38
   macro avg       0.09      0.05      0.06        38
weighted avg       1.00      0.50      0.67        38



In [18]:
df[df.BRAF !='WT']['SUBCLASS'].value_counts().reset_index(name='count')
cdf2 = df[df.BRAF !='WT']['SUBCLASS'].value_counts().reset_index(name='count')
cdf2

,SUBCLASS,count
0,THCA,186
1,SKCM,139
2,COAD,22
3,STES,21
4,LUAD,15
5,LUSC,8
6,UCEC,7
7,GBMLGG,7
8,PRAD,5
9,BRCA,5


In [ ]:
df = pd.read_csv('../input/train.csv')
def cleansing_cells():
    from collections import defaultdict
    ddic = defaultdict()
    for d in df.SUBCLASS.unique():
    ddic[d]=set()
    columlist = df.iloc[:,2:].columns
    
    cnt_droped_genes = 0 
    cnt_cleansed = 0
    for gene in columlist: 
        #print(gene)
        cdf2 = df[df[gene] !='WT']['SUBCLASS'].value_counts().reset_index(name='count')
        sum_count = cdf2['count'].sum()
        data = np.array(cdf2['count'].values).reshape(-1, 1)
        
        if sum_count <= 25:
            print(gene)
            df[gene]='WT'
            #행삭제
            #df.drop(gene, axis=1, inplace=True)
            cnt_droped_genes += 1
            continue
        elif sum_count > 25 and sum_count<100:
            #0으로 클렌징
            df[gene]='WT'
            cnt_cleansed+=1
            #print("mid")
            #print(cdf2)
            #kmeans = KMeans(n_clusters=2, random_state=42)
            #kmeans.fit(data)
            #labels = np.array(kmeans.labels_)
            #print(labels[0])
            #positions = np.where(labels ==labels[0])[0]
            #print(positions)
        
    print(cnt_droped_genes)
    print(cnt_cleansed)


In [125]:
len(ddic)

26

In [128]:
for dict in ddic:
    print(len(ddic[dict]))
    selected_gene_df = pd.DataFrame ({'selected_genes' : list(ddic[dict])})
    selected_gene_df.to_csv('selected_'+dict+'.csv',index=False)

181
280
409
286
218
413
158
212
247
136
338
294
268
349
95
225
300
272
179
169
178
142
110
175
242
292


In [5]:
df_test[df_test['A2M'].apply(lambda x : str(x).startswith('N'))]

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
793,N867Y,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1141,N1196D N1296D N1146D,A473T A506T,A352V A356V,D173E,WT,P623P,R214R R184R,P388S,T897T,I397F,...,A1506A A1366A,WT,WT,WT,WT,WT,WT,WT,R203Q,L556M L525M
1519,N333D N383D N483D,WT,WT,WT,WT,A748V,WT,WT,WT,WT,...,WT,WT,WT,WT,S1198S,WT,WT,WT,Q29X,WT
1576,N991S N841S N891S,WT,WT,A910A,WT,WT,R764Q R734Q,WT,L1894L,WT,...,WT,WT,WT,WT,WT,R163H,N300N N299N,WT,WT,WT


In [6]:
df[df['A2M'].apply(lambda x : str(x).startswith('N'))]

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
594,TRAIN_0594,CESC,N124N,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3432,TRAIN_3432,STES,N1038T,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4913,TRAIN_4913,STES,N1413K,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [359]:
df[df.ABCB8 !='WT']['SUBCLASS'].value_counts()

SUBCLASS
STES      10
SKCM       7
LUAD       6
COAD       5
BRCA       5
UCEC       4
CESC       3
BLCA       3
DLBC       2
OV         2
TGCT       2
LUSC       1
ACC        1
GBMLGG     1
KIRC       1
KIPAN      1
PAAD       1
SARC       1
Name: count, dtype: int64

In [314]:
data[data.columns[0*2]]

0          WT
1          WT
2       R895R
3          WT
4          WT
        ...  
6196       WT
6197       WT
6198       WT
6199       WT
6200       WT
Name: A2M, Length: 6201, dtype: object

In [315]:
data[data.columns[2]]

0       WT
1       WT
2       WT
3       WT
4       WT
        ..
6196    WT
6197    WT
6198    WT
6199    WT
6200    WT
Name: AADAT, Length: 6201, dtype: object

In [305]:
data.loc[:, data.columns[2:4]]

,AADAT,AARS1
0,WT,WT
1,WT,WT
2,WT,WT
3,WT,WT
4,WT,WT
...,...,...
6196,WT,WT
6197,WT,WT
6198,WT,WT
6199,WT,WT


In [276]:
df

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,TRAIN_6197,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,TRAIN_6198,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,TRAIN_6199,TGCT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,SARC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TRAIN_0002,SKCM,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,KIRC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAIN_0004,GBMLGG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,TRAIN_6196,LUAD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,TRAIN_6197,LGG,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,TRAIN_6198,COAD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,TRAIN_6199,TGCT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [224]:
glist = list(df.columns)
len(glist)
    

4386

In [190]:
data.loc[:, 'A2M']


0       0
1       0
2       1
3       0
4       0
       ..
6196    0
6197    0
6198    0
6199    0
6200    0
Name: A2M, Length: 6201, dtype: int64

In [197]:
data.loc[:, data.columns[0:3]]

,A2M,AAAS,AADAT
0,0,0,0
1,0,0,0
2,1,0,0
3,0,0,0
4,0,0,0
...,...,...,...
6196,0,0,0
6197,0,0,0
6198,0,0,0
6199,0,0,0


In [280]:
df.loc[:, df.columns[2:5]].agg(sum)

A2M      148
AAAS      44
AADAT     25
dtype: object

In [201]:
data.loc[:, data.columns[0:4]].agg(sum)

A2M      148
AAAS      44
AADAT     25
AARS1      0
dtype: int64

In [207]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
df[df.SUBCLASS == 'KIPAN']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
16,TRAIN_0016,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
17,TRAIN_0017,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
59,TRAIN_0059,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
66,TRAIN_0066,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132,TRAIN_6132,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,S196G,WT,WT,WT,WT,WT
6137,TRAIN_6137,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6163,TRAIN_6163,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6186,TRAIN_6186,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [215]:
target.loc[:, data.columns[0:1]].agg(sum) 

A2M    WTWTWTWTWTWTWTWTWTWTWTWTS660RWTWTWTWTWTWTWTWTW...
dtype: object

In [ ]:
ndf = pd.DataFrame()
for i in range(0, 1): #len(glist)):
    if i != 0 or i != len(glist)-1:
        ndf[]
        #ndf[glist[i]] =
        
        
        
    #print(i, glist[i])


array(['KIPAN', 'SARC', 'SKCM', 'KIRC', 'GBMLGG', 'STES', 'BRCA', 'THCA',
       'LIHC', 'HNSC', 'PAAD', 'OV', 'PRAD', 'UCEC', 'LAML', 'COAD',
       'ACC', 'LGG', 'LUSC', 'LUAD', 'CESC', 'PCPG', 'THYM', 'BLCA',
       'TGCT', 'DLBC'], dtype=object)

In [61]:
df[df.SUBCLASS=='ACC']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
30,TRAIN_0030,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,TRAIN_0044,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,TRAIN_0050,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183,TRAIN_0183,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
656,TRAIN_0656,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5901,TRAIN_5901,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5917,TRAIN_5917,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6035,TRAIN_6035,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6082,TRAIN_6082,ACC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
df[df.SUBCLASS=='ACC']['PLEC']

30                    A2113A L1321L
44             A2113A A2106A K2047*
50                               WT
183            A2113A A2106A L1321L
656                              WT
                   ...             
5901                             WT
5917    A2113A A2106A A1697A L1321L
6035                         A2507A
6082                         A2495A
6148                             WT
Name: PLEC, Length: 72, dtype: object

In [69]:
da = df[df.SUBCLASS=='LAML']
cnt = []
for c in da.columns[2:]:
        cnt.append(da[c].apply(lambda x : x!=0).agg(sum))
max(cnt)

41

In [62]:

count_df = df[df.SUBCLASS=='LAML'].apply(lambda x : x!=0).agg(sum).sort_values(ascending=False).reset_index(name='count')
print(count_df)
total_rows = count_df[count_df['index']== 'ID']['count'].values[0]
print(count_df[count_df['count'] >= total_rows*0.75][2:])



#print(count_df[count_df['count']]>count_df['ID'])
#gene_list = count_df[count_df['count']>0]['index'].values
#print(len(gene_list))
#gene_list

         index  count
0           ID    158
1     SUBCLASS    158
2         NPM1     41
3         IDH1     18
4         IDH2     17
...        ...    ...
4381     ZNRF4      0
4382       A2M      0
4383      ZW10      0
4384     ZWINT      0
4385       ZYX      0

[4386 rows x 2 columns]
Empty DataFrame
Columns: [index, count]
Index: []


In [50]:
df[df.SUBCLASS=='SKCM'].apply(lambda x : x!='WT').columns


Index(['ID', 'SUBCLASS', 'A2M', 'AAAS', 'AADAT', 'AARS1', 'ABAT', 'ABCA1',
       'ABCA2', 'ABCA3',
       ...
       'ZNF292', 'ZNF365', 'ZNF639', 'ZNF707', 'ZNFX1', 'ZNRF4', 'ZPBP',
       'ZW10', 'ZWINT', 'ZYX'],
      dtype='object', length=4386)

In [74]:
def get_gene_list2():
    genes = []
    gene_set = set()
    for disease in df["SUBCLASS"].unique():
        disease_df = df[df["SUBCLASS"] == disease]

        non_wt_counts = (disease_df != 'WT').sum()  # "WT"가 아닌 행 개수
        non_wt_counts_sorted = non_wt_counts.sort_values(ascending=False) #빈도 수가 높은 순서대로 정렬

        n = 20 # 추출한 상위 유전자 개수
        genes.extend(list(non_wt_counts_sorted[2:2+n].index))

        #print(f'{disease} - 전체 행의 개수 : {len(disease_df)}')
        dd = non_wt_counts_sorted[2:n].reset_index()
        gene_list1 = dd['index'].values
        gene_set.update(gene_list1)
        #print(gene_list1)
    return gene_set

In [8]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [10]:
df.loc[:, df.columns[1:2]]

,SUBCLASS
0,KIPAN
1,SARC
2,SKCM
3,KIRC
4,GBMLGG
...,...
6196,LUAD
6197,LGG
6198,COAD
6199,TGCT


In [14]:
df_ = data.copy()
df_['SUBCLASS']= df.loc[:, df.columns[1:2]]
df_
    

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX,SUBCLASS
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIPAN
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,SARC
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,SKCM
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,KIRC
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,LUAD
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,LGG
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,T181S,WT,COAD
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,TGCT


In [6]:
#### ####

,disease,counts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [23]:
df[df.SUBCLASS == 'KIPAN'].shape[0]

515

In [12]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6197,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6198,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,T181S,WT
6199,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [26]:
selected_genes = set()
for d in disease_list:
    df_ = data.copy()
    df_['SUBCLASS'] = df.loc[:, df.columns[1:2]]
    count_df = df_[df_.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
    count_df = count_df[2:].reset_index(name='count')        
    num_transaction = diseaseList[diseaseList.disease == d]['counts'].values[0]
    th = 20 if num_transaction > 250 else 5
    #th = num_transaction//divider
    #print( num_transaction, th)
    
    gene_list = count_df[count_df['count']>th]['index'].values
    print(d, len(gene_list))
    selected_genes.update(gene_list)
print(len(selected_genes))

    

KIPAN 8
SARC 32
SKCM 372
KIRC 2
GBMLGG 13
STES 221
BRCA 25
THCA 2
LIHC 107
HNSC 334
PAAD 23
OV 2
PRAD 2
UCEC 2006
LAML 10
COAD 936
ACC 52
LGG 17
LUSC 587
LUAD 468
CESC 176
PCPG 4
THYM 2
BLCA 209
TGCT 15
DLBC 18
2226


In [27]:
selected_genes.remove("SUBCLASS")

In [28]:
total = pd.concat([data,df_test] ) #make_ndf(data, df_test)
data = total[:6201]
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
2544,WT,WT,WT,WT,WT,WT,WT,WT,R1517H,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [29]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [30]:

total = total.loc[:, list(selected_genes)]
total

,NDRG2,PTK2,SERPINA5,ST14,ISG20L2,MAPK8,SORBS2,SF3A1,STX3,ARFGEF2,...,MTA1,KRT5,PREX1,ERBB2,PDIA5,PCM1,PLK3,MYH8,ZNF112,MDH1
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,WT,WT,WT,WT,WT,Y266N,WT,WT,WT,WT,...,WT,G104R,WT,WT,WT,WT,WT,F492F,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,I157I I111I I171I I167I,R293H R450H R272H R406H R419H R370H,WT,P362P,P48Lfs,WT,V91V V160V V177V V137V V10V V270V V191V,P637S,E167E,Q1776H,...,Q403R,I467N,Y1192X,G867R G865R G866R G795R G819R G605R G880R G891...,F484F,D1765E D1672E D1673E D1668E D1703E D1665E D171...,WT,R1178C,WT,WT
2544,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [31]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for c in total.columns:
    total[c] = encoder.fit_transform(total[c])



In [32]:
df = total[:6201]
df_test = total[6201:]



In [33]:
df

,NDRG2,PTK2,SERPINA5,ST14,ISG20L2,MAPK8,SORBS2,SF3A1,STX3,ARFGEF2,...,MTA1,KRT5,PREX1,ERBB2,PDIA5,PCM1,PLK3,MYH8,ZNF112,MDH1
0,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
1,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
2,59,135,75,110,50,59,192,87,44,233,...,90,35,261,167,75,154,72,118,119,48
3,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
4,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,59,135,75,110,50,58,192,87,44,233,...,90,122,261,34,75,154,72,294,119,48
6197,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
6198,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
6199,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48


In [34]:
df_test

,NDRG2,PTK2,SERPINA5,ST14,ISG20L2,MAPK8,SORBS2,SF3A1,STX3,ARFGEF2,...,MTA1,KRT5,PREX1,ERBB2,PDIA5,PCM1,PLK3,MYH8,ZNF112,MDH1
0,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
1,59,135,75,110,50,58,192,87,44,227,...,90,122,261,167,75,154,72,340,119,49
2,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
3,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
4,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
2542,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48
2543,25,84,75,59,30,58,190,57,10,151,...,57,58,262,47,19,12,72,240,119,48
2544,59,135,75,110,50,58,192,87,44,233,...,90,122,261,167,75,154,72,340,119,48


In [35]:
from sklearn.preprocessing import LabelEncoder
X = df
y = df_target
encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)
y


array([ 8, 19, 20, ...,  4, 22, 20])

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)#shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 2225) (1861, 2225)


In [48]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)


#Multi-class classification

In [49]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.62824	valid-mlogloss:2.71480
[2]	train-mlogloss:2.24031	valid-mlogloss:2.45671
[4]	train-mlogloss:2.01554	valid-mlogloss:2.33410
[6]	train-mlogloss:1.85673	valid-mlogloss:2.25956
[8]	train-mlogloss:1.73832	valid-mlogloss:2.21051
[10]	train-mlogloss:1.64588	valid-mlogloss:2.17373
[12]	train-mlogloss:1.57216	valid-mlogloss:2.15253
[14]	train-mlogloss:1.51054	valid-mlogloss:2.13558
[16]	train-mlogloss:1.45816	valid-mlogloss:2.12330
[18]	train-mlogloss:1.41052	valid-mlogloss:2.11760
[20]	train-mlogloss:1.37000	valid-mlogloss:2.11572
[22]	train-mlogloss:1.33327	valid-mlogloss:2.10998
[24]	train-mlogloss:1.30005	valid-mlogloss:2.11064
[26]	train-mlogloss:1.26837	valid-mlogloss:2.10728
[28]	train-mlogloss:1.24046	valid-mlogloss:2.10679
[30]	train-mlogloss:1.21388	valid-mlogloss:2.10766
[32]	train-mlogloss:1.18919	valid-mlogloss:2.10731
[34]	train-mlogloss:1.16606	valid-mlogloss:2.10810
[36]	train-mlogloss:1.14478	valid-mlogloss:2.10801
[37]	train-mlogloss:1.13429	valid-ml

In [50]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'learning_rate': 0.05
}
num_boost_round = 200

clf = xgb.XGBClassifier(n_estimators=num_boost_round, early_stopping_rounds=5, **params) 
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.14140	validation_1-mlogloss:3.15612
[2]	validation_0-mlogloss:2.96646	validation_1-mlogloss:3.00736
[4]	validation_0-mlogloss:2.83165	validation_1-mlogloss:2.89755
[6]	validation_0-mlogloss:2.72221	validation_1-mlogloss:2.81219
[8]	validation_0-mlogloss:2.62965	validation_1-mlogloss:2.74058
[10]	validation_0-mlogloss:2.54834	validation_1-mlogloss:2.67980
[12]	validation_0-mlogloss:2.47601	validation_1-mlogloss:2.62843
[14]	validation_0-mlogloss:2.41171	validation_1-mlogloss:2.58304
[16]	validation_0-mlogloss:2.35331	validation_1-mlogloss:2.54256
[18]	validation_0-mlogloss:2.29990	validation_1-mlogloss:2.50764
[20]	validation_0-mlogloss:2.25143	validation_1-mlogloss:2.47644
[22]	validation_0-mlogloss:2.20583	validation_1-mlogloss:2.44799
[24]	validation_0-mlogloss:2.16362	validation_1-mlogloss:2.42248
[26]	validation_0-mlogloss:2.12451	validation_1-mlogloss:2.39962
[28]	validation_0-mlogloss:2.08778	validation_1-mlogloss:2.37889
[30]	validation_0-mlogloss:2.0

In [51]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1861,), (1861,), (1861, 26))

In [52]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.34712520150456744

In [89]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.3449758194519076

In [50]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.78      0.82        23
           1       0.29      0.06      0.10        35
           2       0.25      0.68      0.37       238
           3       0.14      0.03      0.04        39
           4       0.63      0.67      0.65        63
           5       0.75      0.38      0.50        16
           6       0.34      0.29      0.31       150
           7       0.34      0.25      0.29        60
           8       0.28      0.32      0.30       159
           9       0.10      0.04      0.06       104
          10       0.74      0.29      0.42        48
          11       0.29      0.49      0.36        57
          12       0.35      0.19      0.24        48
          13       0.19      0.05      0.08        59
          14       0.19      0.14      0.16        49
          15       0.26      0.48      0.33        93
          16       0.14      0.04      0.06        28
          17       0.30    

In [53]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8585507419794566

In [54]:
clf.predict(df_test)

array([21, 25,  2, ..., 25, 10,  6])

In [55]:
res = clf.predict(df_test)



In [56]:
res = encoder2.inverse_transform(res)

In [57]:

df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,HNSC
...,...,...
2541,TEST_2541,BLCA
2542,TEST_2542,HNSC
2543,TEST_2543,UCEC
2544,TEST_2544,LAML


In [58]:
res_df.to_csv('res08-3.csv',index = False)

In [97]:
pred_probs = xgb_model.predict(dtest)


NameError: name 'xgb_model' is not defined

In [ ]:
import numpy as np
print(np.round(pred_probs[:10], 3))


In [58]:
preds= [round(x) if x > 0 else 0 for x in pred_probs]

In [ ]:
preds

In [60]:
for p in preds:
    if p<0:
        print(p)


In [ ]:
encoder.inverse_transform(preds)

In [35]:
model = RandomForestClassifier(
            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            criterion='entropy', ## 불순도 측도
            max_depth=65, ## 개별 나무의 최대 깊이
            #max_features='sqrt', ## 매 분리시 랜덤으로 뽑을 변수 개수
            #max_samples=1.0, ## 붓스트랩 샘플 비율 => 1이면 학습데이터를 모두 샘플링한다.
            #bootstrap=True, ## 복원 추출,  False이면 비복원 추출
            oob_score=True, ## Out-of-bag 데이터를 이용한 성능 계산
            random_state=100
).fit(X,y)


In [ ]:
## 예측
print(model.predict(X)[:3]) 
print()


In [ ]:
X[:3]

In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
## 성능 평가
print(model.oob_score_) ## Out-of-bag 성능 평가 점수
print('정확도 : ', model.score(X,y)) ## 테스트 성능 평가 점수(Accuracy)
print()
## 변수 중요도
print(model.feature_importances_)




In [ ]:
res = model.predict(df_test)
res

In [ ]:
res = model.predict(df_test)
res

In [ ]:
df_test1 = pd.read_csv('test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df



In [24]:
res_df.to_csv('res04.csv',index = False)

In [ ]:
df[df.SUBCLASS == 'BRCA'].sum().sort_values(ascending=False).str.replace('WT','')

In [ ]:
df[df['SUBCLASS']=='DLBC']

In [ ]:
import numpy as np 
dfs = df[df['SUBCLASS']=='DLBC']
dfs.columns

In [ ]:
for c in dfs.columns:
    dfs[c] = np.where(dfs[c] == 'WT', 0, 1)
dfs

    

In [ ]:
dfs = df[df['SUBCLASS']=='DLBC']
dfs.apply(lambda x: x(if x=='WT' else x=1))

In [ ]:
for  x in df.iloc[0]:
    if x != 'WT': print(x)


In [ ]:
len(df_test.columns)

In [ ]:
len(df_test)

In [ ]:
df.colu